In [ ]:
import requests
import getpass
from datetime import datetime as dt
from datetime import timedelta
from IPython.display import clear_output

from arcgis.gis import GIS

## Setup your connection to ArcGIS Online

In [ ]:
username = input('Username: ')
password = getpass.getpass('Password: ')
arcgis_url = 'https://www.arcgis.com'

# create your `gis` instance 
gis = GIS(arcgis_url, username, password)

print ('succesfully logged in.')

## Setup the request to the API

In [ ]:
# get events from <x> days ago
look_back_days = 35

days_ago_dte = dt.now() - timedelta(days=look_back_days)
days_ago = days_ago_dte.strftime('%Y-%m-%d')

# you can limit the number of records that return from the API
# set to 0 for no limit
limit = 10

api_url = 'https://api.acleddata.com/acled/read?event_date={}&event_date_where=%3E=&limit={}'.format(days_ago, limit)

## Request data from the ACLED API

In [ ]:
response = requests.get(api_url)
data = response.json()
rows = data['data']
print ('ACLED API returned ~ {} ~ rows'.format(len(rows)))

## ItemId of ArcGIS Online/Portal ACLED Live service

In [ ]:
item_id = '910a2cd8097349a1bb11de437e830f5d'
live_item = gis.content.get(itemid=item_id)
fl = live_item.layers[0]

## Delete existing features

In [ ]:
try:
    delete_response = fl.delete_features(where='1=1')
    delete_results = delete_response['deleteResults']
    print ('succesfully deleted {} rows'.format(len(delete_results)))
except Exception as e:
    print ('error deleting features')

## construct the features

In [ ]:
features = []
for event in rows:
    feature = {
        'geometry': {
            'x': float(event['longitude']),
            'y': float(event['latitude'])
        },
        'attributes': {
            'data_id': int(event['data_id']),
            'iso': event['iso'],
            'event_id_cnty': event['event_id_cnty'],
            'event_id_no_cnty': event['event_id_no_cnty'],
            'event_date': event['event_date'],
            'year': int(event['year']),
            'time_precision': event['time_precision'],
            'event_type': event['event_type'],
            'actor1': event['actor1'],
            'assoc_actor_1': event['assoc_actor_1'],
            'inter1': event['inter1'],
            'actor2': event['actor2'],
            'assoc_actor_2': event['assoc_actor_2'],
            'inter2': event['inter2'],
            'interaction': event['interaction'],
            'region': event['region'],
            'country': event['country'],
            'admin1': event['admin1'],
            'admin2': event['admin2'],
            'admin3': event['admin3'],
            'location': event['location'],
            'latitude': float(event['latitude']),
            'longitude': float(event['longitude']),
            'geo_precision': event['geo_precision'],
            'source': event['source'],
            'source_scale': event['source_scale'],
            'notes': event['notes'],
            'fatalities': int(event['fatalities']),
            'timestamp': int(event['timestamp']),
            'iso3': event['iso3']
        }
    }

    features.append(feature)

## Chunk up the features to be added
When using feature services with a large amount of features, it's sometimes best to chunk up any editing jobs into reasonable sized requests. Here we will add 1,000 features at a time.

In [ ]:
n =  1000
feature_chunks = [features[i:i+n] for i in range(0, len(features), n)]

## Make the edits to the feature layer
Here we loop through each chunk of 1,000 features and apply those edits

In [ ]:
c_len = len(feature_chunks)
for i, adds in enumerate(feature_chunks):
    try:
        fl.edit_features(adds=adds)
        print ('added chunk {} of {}'.format(i+1, c_len))
        clear_output(wait=True)
    except Exception as e:    
        print ('error adding chunk')

print ('done')